# Initialization

## Importing libs and setting plot style

In [3]:
import itertools
import multiprocessing as mp
import os
import random
import threading
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from atpbar import atpbar, flush
from matplotlib import rc

In [2]:
# Setting plot style

sns.set()
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.0})

rc("text", usetex=True)

sns.set_style("ticks")
sns.set_style(
    "whitegrid",
    {
        "axes.edgecolor": "black",
        "axes.grid": True,
        "axes.axisbelow": True,
        "axes.labelcolor": ".15",
        "grid.color": "0.9",
        "grid.linestyle": "-",
        "xtick.direction": "in",
        "ytick.direction": "in",
        "xtick.bottom": True,
        "xtick.top": True,
        "ytick.left": True,
        "ytick.right": True,
        "font.family": ["sans-serif"],
        "font.sans-serif": ["Liberation Sans", "Bitstream Vera Sans", "sans-serif"],
    },
)

## Global variables

In [4]:
# folder with stored data
data_folder = "../../data"

# MH parameters
spins = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6]
sigmas = [0.4, 0.35, 0.35, 0.35, 0.35, 0.35, 0.32, 0.3]
lengths = [1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000]
burnins = [10, 10, 10, 10, 10, 10, 10, 10]
chains = [30, 30, 30, 30, 30, 30, 30, 30]

# set optimal number of threads
optimal_number_of_threads = int(mp.cpu_count())
print(f"optimal number of threads: {optimal_number_of_threads}")

optimal number of threads: 12


# Angles

## Computation

In [5]:
# takes an intertwiner and returns the corresponding angle eigenvalue
def from_intertwiner_to_angle(matrix_element, spin):
    return (matrix_element * (matrix_element + 1) - 2 * spin * (spin + 1)) / (
        2 * spin * (spin + 1)
    )

In [6]:
def from_draws_to_angles(
    folder_prefix,
    spin,
    length,
    burnin,
    angle_path,
    chain_id,
    n,
    name,
    lag_max_autocorr=8000,
    node_autocorr=1,
):

    for i in atpbar(range(n), name=name):
        time.sleep(0.0001)

    draw_path = f"{folder_prefix}/draws/draws_chain_{chain_id}.csv"

    # load in memory the stored draws
    df = pd.read_csv(draw_path, low_memory=False)
    df.columns = df.columns.str.replace("intertwiner ", "node ", regex=True)
    df.columns = df.columns.str.replace("multeplicity", "multiplicity", regex=True)

    # retrieving relevant parameters
    multiplicity = df[["draw multiplicity"]].to_numpy().astype(int)
    total_accept_draws = int(df["total accept. draws"][0])
    total_accept_rate = float(df["total accept. rate"][0].strip("%"))
    total_run_time = float(df["total run time"][0].strip(" s"))

    # dropping columns (leaving multiplicity)
    df = df.drop(
        columns=[
            "draw amplitude",
            "total accept. draws",
            "total accept. rate",
            "total run time",
        ]
    )

    # from intertwiners to angles
    df.iloc[:, :16] = df.iloc[:, :16].apply(from_intertwiner_to_angle, args=(spin,))

    # taking multiplicity into account
    df = df.loc[df.index.repeat(df["draw multiplicity"])]

    # dropping multiplicity column
    df = df.drop(
        columns=[
            "draw multiplicity",
        ]
    )

    # remove some more iterations as burn-in phase
    # (I do this as I think b = 10 is too low)
    # additional_burnin = int(490)
    # df.drop(index=df.index[:additional_burnin], axis=0, inplace=True)

    ##########################################################
    # Computing autocorrelation (acf)
    ##########################################################

    acf_node = sm.tsa.acf(df[f"node {node_autocorr}"], nlags=lag_max_autocorr)
    df_final = pd.DataFrame(acf_node, columns=[f"node {node_autocorr}"])
    df_final.index.names = ["lag"]

    angle_autocorr_path_chain = (
        f"{angle_path}/autocorrelations/node_{node_autocorr}_chain_{chain_id}.csv"
    )
    df_final.to_csv(angle_autocorr_path_chain, index=True)

    ##########################################################
    # Computing exp values (avg) and quantum spread (std dev)
    ##########################################################

    df_final = pd.concat([df.mean(), df.std()], axis=1)
    df_final.columns = ["cosine angle avg", "cosine angle std"]

    angle_exp_values_path_chain = f"{angle_path}/exp_values/chain_{chain_id}.csv"
    df_final.to_csv(angle_exp_values_path_chain, index=True)

    ##########################################################
    # Computing correlations (pearson corr)
    ##########################################################

    df_final = df.corr(method="pearson")

    angle_corr_path_chain = f"{angle_path}/correlations/chain_{chain_id}.csv"
    df_final.to_csv(angle_corr_path_chain, index=True)

In [7]:
# Converts multiple draws into angles exp. values, quantum spread, correlations and autocorrelations.
# Store the result for each batch, then combines all batchs in another CSV file
def angles_compute(
    data_folder,
    spin,
    length,
    batch_size,
    sigma,
    burnin,
    number_of_threads=optimal_number_of_threads,
    node_autocorr=1,
):

    number_of_batches = length/batch_size
    
    folder_prefix = f"Iterations_{length}_batch_size_{batch_size}_burnin_{burnin}_sigma_{sigma}/j_{spin}"
    batch_id_collection = []

    for batch_id in range(1, number_of_batches + 1):
        draw_path = f"{folder_prefix}/draws/draws_batch_n_{batch_id}.csv"
        if os.path.isfile(draw_path):
            batch_id_collection.append(batch_id)
        else:
            warnings.warn("Warning: the draw %s was not found" % (draw_path))

    batches_to_assemble = len(batch_id_collection)

    if batches_to_assemble != 0:

        angle_path = f"{folder_prefix}/operators/angles"

        angle_path_exp_values = f"{angle_path}/exp_values"
        angle_path_autocorrelations = f"{angle_path}/autocorrelations"

        os.makedirs(f"{angle_path_exp_values}", exist_ok=True)
        os.makedirs(f"{angle_path_autocorrelations}", exist_ok=True)

        print(
            f"\nComputing exp. values and autocorrelations of {batches_to_assemble} draws batches, using {number_of_threads} threads...\n"
        )

        print(
            f"\nspin = {spin}, length = {length}, sigma = {sigma}, length = {length}, burnin = {burnin}, node_autocorr = {node_autocorr}\n"
        )

        threads = []
        for chain_id in chain_id_collection:

            name = "chain {}".format(chain_id)
            n = random.randint(number_of_threads, 10000)

            t = threading.Thread(
                target=from_draws_to_angles,
                args=(
                    folder_prefix,
                    spin,
                    length,
                    burnin,
                    angle_path,
                    chain_id,
                    n,
                    name,
                ),
            )
            threads.append(t)
            t.start()

        # wait for the threads to complete
        for t in threads:
            t.join()

        flush()

        print(f"All draws have been processed")

        print(f"\nAssembling {chains_to_assemble} chains...")

        ##########################################################
        # Assembling autocorrelation
        ##########################################################

        DF_list = [
            pd.read_csv(
                f"{angle_path}/autocorrelations/node_{node_autocorr}_chain_{chain_id+1}.csv",
                index_col=0,
                low_memory=False,
            )
            for chain_id in range(chains_to_assemble)
        ]

        df_all_chains = pd.concat(DF_list[:]).groupby(level=0, sort=False)

        df_final = pd.concat([df_all_chains.mean()], axis=1)
        df_final.columns = ["node 1"]

        df_final.to_csv(
            f"{angle_path_autocorrelations}/chains_assembled_{chains_to_assemble}.csv",
            index=True,
        )

        ##########################################################
        # Assembling exp values and quantum spread
        ##########################################################

        DF_list = [
            pd.read_csv(
                f"{angle_path_exp_values}/chain_{chain_id+1}.csv",
                index_col=0,
                low_memory=False,
            )
            for chain_id in range(chains_to_assemble)
        ]

        df_all_chains = pd.concat(DF_list[:]).groupby(level=0, sort=False)

        df_tmp = df_all_chains.std()
        df_tmp.drop(columns=df_tmp.columns[-1], axis=1, inplace=True)
        df_tmp.columns = ["cos avg std over chains"]

        df_final = pd.concat([df_all_chains.mean(), df_tmp], axis=1)
        df_final.columns = ["cos avg", "cos std", "cos avg std over chains"]
        df_final.T

        df_final.to_csv(
            f"{angle_path_exp_values}/chains_assembled_{chains_to_assemble}.csv",
            index=True,
        )

        ##########################################################
        # Assembling correlations
        ###########################################################

        DF_list = [
            pd.read_csv(
                f"{angle_path_correlations}/chain_{chain_id+1}.csv",
                index_col=0,
                low_memory=False,
            )
            for chain_id in range(chains_to_assemble)
        ]

        df_all_chains = pd.concat(DF_list[:]).groupby(level=0, sort=False)

        df_final = pd.concat([df_all_chains.mean()], axis=1)

        df_final.to_csv(
            f"{angle_path_correlations}/chains_assembled_{chains_to_assemble}.csv",
            index=True,
        )

        print("Done")

    else:
        warnings.warn("I can't compute angles since there are no chains available")

In [ ]:
for spin, sigma, length, burnin, chains_to_assemble in zip(
    spins, sigmas, lengths, burnins, chains
):

    angles_compute(data_folder, spin, length, sigma, burnin, chains_to_assemble)